In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import cloveFunctions as clv
import matplotlib
# matplotlib.use('Qt4Agg')
import matplotlib.pyplot as plt
# matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
import warnings
warnings.filterwarnings('ignore')

## why so many -- in when computing matchedpairs?
its because pos and neg could be all nan!

In [32]:
lungresults = pd.read_csv('data/tcga_v_ccle_lung.tab.gz', sep='\t', compression='gzip', index_col=0)
lungmatch = pd.read_csv('data/tcga_lung_cloves.tab.gz', compression='gzip', sep='\t', index_col=0)

In [4]:
lungresults.head()

,exp,cnv,t_tcga_lung,t_null_tcga_lung,p_tcga_lung,t_ccle_lung,p_ccle_lung
430,FOXH1,METTL2B,0.718796,0.326838,0.472436,--,--
431,FOXH1,MYOM1,-0.898618,-0.719695,0.369074,--,--
432,FOXH1,CCDC148,-0.130872,1.082172,0.895903,--,--
433,FOXH1,IRGQ,0.158604,1.022911,0.874014,--,--
435,FOXH1,DNAJC5B,2.809684,-0.841591,0.005057,--,--


In [28]:
lungresults.tail()

,exp,cnv,t_tcga_lung,t_null_tcga_lung,p_tcga_lung,t_ccle_lung,p_ccle_lung
999135,EPCAM,CUL9,3.074485,-0.380574,2.166397e-03,NaN,NaN
999136,EPCAM,CPA4,2.916369,-0.231343,3.621154e-03,-1.44808652549,0.149467682963
999138,EPCAM,FITM1,0.906148,0.628090,3.650778e-01,NaN,NaN
999142,EPCAM,HRSP12,0.098356,0.938029,9.216694e-01,NaN,NaN
999144,EPCAM,OSM,-5.432958,0.283584,6.976084e-08,0.580590573831,0.562299060671


In [38]:
bait = 'METTL2B'
prey = 'FOXH1'

pos = np.array(expdf.loc[prey][cmask.loc[bait]])
neg = np.array(expdf.loc[prey][~cmask.loc[bait]])

t, p = stats.ttest_ind(pos, neg, nan_policy=nan_style, equal_var=True)
print(t,type(t), p, type(p))

print(pos,(~np.isnan(pos)).sum(), neg,(~np.isnan(neg)).sum())

-- <class 'numpy.ma.core.MaskedConstant'> -- <class 'numpy.ma.core.MaskedConstant'>
[ nan  nan  nan  nan  nan] 0 [      nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan       nan       nan       nan       nan
       nan       nan       nan     

In [39]:
bait = 'CPA4'
prey = 'EPCAM'

pos = np.array(expdf.loc[prey][cmask.loc[bait]])
neg = np.array(expdf.loc[prey][~cmask.loc[bait]])

t, p = stats.ttest_ind(pos, neg, nan_policy=nan_style, equal_var=True)
print(t,type(t), p, type(p))

print(pos,(~np.isnan(pos)).sum(), neg,(~np.isnan(neg)).sum())

-1.44808652549 <class 'numpy.float64'> 0.149467682963 <class 'numpy.float64'>
[  4.868861  13.49004    8.777356        nan] 3 [ 10.42923   12.98001   12.90409   12.8771    10.31661   13.25583   13.44435
  12.88363   13.29148    5.986706  12.96776    7.616331  10.58463   11.55726
  13.09149   13.42589    7.758206  13.27937   13.15714    9.677115
  12.78076   11.27572    9.683023  12.51106   13.03133   12.17999   10.31708
  13.27034    4.721761   5.222609  12.87664         nan  12.62039   12.14324
  12.7247    13.39787   13.1009    13.03704    9.612444  11.31807
   6.314472  13.09115   11.25794    6.572896  12.87214    6.51735   12.41995
  12.20619   10.71551   13.25583   12.50343   12.75253    6.202319
  12.62209         nan  12.35745   12.20425   12.54176   12.44365
   8.693022  11.50725   12.63042   12.94446   13.29013   12.82851   11.8369
  12.59465    5.963532  12.83011   10.90006   13.11349    6.272704
   7.315222   7.497744  12.99763    4.616601  12.83346   12.61842   13.16235
  1

In [50]:
min_var, min_n = 2, 2
expdf, cnvdf = clv.load_data('data/lung_mrna_df.pickle', 'data/lung_cnv_df.p', details=False)
expdf, cnvdf = clv.mainFitler(expdf, cnvdf, var=min_var, n=min_n)
matchdf = lungmatch
match_cohort = 'lung'
nan_style='omit'

exp: (17380, 186) --> filter --> (2030, 180)
cnv: (23316, 181) --> filter --> (18312, 180)
37173360 CLOvE pairs are possible with these parameters
estimated calculation time: 433.68919999999997min (~7sec/10k pairs)


In [57]:
np_t_w, np_p_w, np_t_w_null = [], [], []

cells = list(set(cnvdf.columns).intersection(expdf.columns))
expdf = expdf[cells]
cmask = cnvdf[cells] == 1

df = matchdf[['exp', 'cnv', 'np_t_w', 'np_t_w_null','np_p_w']]
df.columns = ['exp', 'cnv', 't_'+match_cohort, 't_null_'+match_cohort, 'p_'+match_cohort]
df = df[(df['exp'].isin(expdf.index)) & (df['cnv'].isin(cnvdf.index))]
df = df.sample(12)
print('attempting {} comparisons referenced from {} (var>{}, n>{}'.format(df.shape[0], match_cohort, min_var, min_n))
  
# progress initialize
count=0
percent_complete=0
comparisons = df.shape[0]

for row in df.itertuples():
    # progress report
    count+=1
    if count%(round(comparisons,-1)/10)==0:
        percent_complete+=10
        print('pair computation {}% complete ({}/{})'.format(percent_complete, count, comparisons))

    # mask cnv contexts onto expression data
    pos = np.array(expdf.loc[row.exp][cmask.loc[row.cnv]])
    neg = np.array(expdf.loc[row.exp][~cmask.loc[row.cnv]])

    if ((~np.isnan(pos)).sum() > min_n) &  ((~np.isnan(neg)).sum() > min_n):
        # calculate t_stat, welch
        t, p = stats.ttest_ind(pos, neg, nan_policy=nan_style, equal_var=True)
        np_t_w.append(t)
        np_p_w.append(p)
    else:    
        np_t_w.append(np.nan)
        np_p_w.append(np.nan)

attempting 12 comparisons referenced from lung (var>2, n>2
pair computation 10% complete (1/12)
pair computation 20% complete (2/12)
pair computation 30% complete (3/12)
pair computation 40% complete (4/12)
pair computation 50% complete (5/12)
pair computation 60% complete (6/12)
pair computation 70% complete (7/12)
pair computation 80% complete (8/12)
pair computation 90% complete (9/12)
pair computation 100% complete (10/12)
pair computation 110% complete (11/12)
pair computation 120% complete (12/12)


## With above problem fixed can we find 3sigma hits?

In [61]:
lungresults = pd.read_csv('data/tcga_v_ccle_lung.tab.gz', sep='\t', compression='gzip', index_col=0)
lungresults.dropna(inplace=True)
print(lungresults.shape)
lungresults.head()

(15755, 7)


,exp,cnv,t_tcga_lung,t_null_tcga_lung,p_tcga_lung,t_ccle_lung,p_ccle_lung
856,BCHE,APOD,2.232715,0.361246,0.025790,-1.011340,0.315547
863,BCHE,TAS2R20,3.081984,-0.981121,0.002113,0.505528,0.614872
865,BCHE,TMEM120A,1.182565,0.333414,0.237265,0.235038,0.814907
880,BCHE,CT45A3,1.159753,-1.322186,0.246429,0.255718,0.798964
905,BCHE,THAP7,4.098205,0.338568,0.000045,1.431856,0.156904


In [80]:
s1_ccle = np.std(lungresults['t_ccle_lung'])
print(s1_ccle)
s1_tcga = np.std(lungresults['t_tcga_lung'])
print(s1_tcga)

1.043068841562431
2.3727201601094543


In [76]:
lungresults[(lungresults['t_tcga_lung']>s3_tcga) & (lungresults['t_ccle_lung']>s3_ccle)]

,exp,cnv,t_tcga_lung,t_null_tcga_lung,p_tcga_lung,t_ccle_lung,p_ccle_lung
905,BCHE,THAP7,4.098205,0.338568,4.504141e-05,1.431856,0.156904
1634,NRCAM,WNT5B,6.242904,0.207698,6.359863e-10,1.942676,0.054733
1661,NRCAM,GP1BB,5.140776,-0.431923,3.295654e-07,1.231436,0.220911
4549,MEST,SLCO1B1,3.239191,-0.388079,1.238545e-03,1.314241,0.190611
4558,MEST,EYA1,3.021683,0.629122,2.578295e-03,1.191593,0.235153
6793,TCEA3,SH2B1,2.598401,2.262073,9.504946e-03,2.034715,0.043595
6799,TCEA3,SPN,2.688134,1.931038,7.305735e-03,2.034715,0.043595
7589,ZNF185,DNAJB11,2.587021,-0.358587,9.822447e-03,1.300293,0.195303
14134,FZD7,TSHZ3,4.406334,0.251572,1.165875e-05,2.105621,0.036733
15157,SMARCA1,PDE4DIP,5.182951,2.513136,2.646310e-07,1.570582,0.118096
